Install + Imports + Chart Saving Setup

Install packages

In [1]:
!pip -q install pandas numpy matplotlib seaborn scikit-learn openpyxl --force-reinstall plotly kaleido

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.34.0 requires jedi>=0.16, which is not installed.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.3.3 which is incompatible.
opencv-contrib-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 2.4.1 which is incompatible.
numba 0.60.0 requires numpy<2.1,>=1.22, but you have numpy 2.4.1 which is incompatible.
tensorflow 2.19.0 requires numpy<2.2.0,>=1.26.0, but you have numpy 2.4.1 which is incompatible.
opencv-python-headless 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 2.4.1 which is incompatible.
gradio 5.50.0 requires pillow<12.0,>=8.0, but you have pillow 12.1.0 which is incompatible.
opencv-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 2.4.1 which is incompatible.


Imports + global saving helper

In [2]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.utils.class_weight import compute_sample_weight

import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio

import os
import warnings
warnings.filterwarnings("ignore")

pio.renderers.default = "colab"

# ---------- Create folder for saving charts ----------
SAVE_DIR = "REPORT_CHARTS"
os.makedirs(SAVE_DIR, exist_ok=True)

def save_fig(fig, filename):
    """
    Saves Plotly fig as PNG using Kaleido.
    Automatically stores in REPORT_CHARTS folder.
    """
    path = os.path.join(SAVE_DIR, filename)
    fig.write_image(path, scale=2)  # Higher quality
    print(f"✅ Saved chart: {path}")

print("✅ Setup complete. Charts will auto-save in REPORT_CHARTS/")

✅ Setup complete. Charts will auto-save in REPORT_CHARTS/


SECTION 1 — Upload + Load Datasets

Cell 1.1 — Upload

In [3]:
from google.colab import files
uploaded = files.upload()

print("✅ Uploaded:")
for k in uploaded.keys():
    print(" -", k)

Saving Aadhaar Biometric Monthly Update Data Maharashtra.csv to Aadhaar Biometric Monthly Update Data Maharashtra.csv
Saving Aadhaar Demographic Monthly Update Data Maharashtra.csv to Aadhaar Demographic Monthly Update Data Maharashtra.csv
Saving Aadhaar Monthly Enrolment data Maharashtra.csv to Aadhaar Monthly Enrolment data Maharashtra.csv
✅ Uploaded:
 - Aadhaar Biometric Monthly Update Data Maharashtra.csv
 - Aadhaar Demographic Monthly Update Data Maharashtra.csv
 - Aadhaar Monthly Enrolment data Maharashtra.csv


Cell 1.2 — Load CSVs

In [4]:
bio_path  = "Aadhaar Biometric Monthly Update Data Maharashtra.csv"
demo_path = "Aadhaar Demographic Monthly Update Data Maharashtra.csv"
enr_path  = "Aadhaar Monthly Enrolment data Maharashtra.csv"

bio_df  = pd.read_csv(bio_path)
demo_df = pd.read_csv(demo_path)
enr_df  = pd.read_csv(enr_path)

print("✅ Loaded datasets:")
print("Biometric:", bio_df.shape)
print("Demographic:", demo_df.shape)
print("Enrolment:", enr_df.shape)

bio_df.head(3)

✅ Loaded datasets:
Biometric: (424731, 6)
Demographic: (93184, 7)
Enrolment: (183245, 6)


,date,state,district,pincode,bio_age_5_17,bio_age_17_
0,28-12-2025,Maharashtra,Ratnagiri,415608,5,8
1,28-12-2025,Maharashtra,Ratnagiri,415619,1,3
2,28-12-2025,Maharashtra,Ratnagiri,415703,3,3


SECTION 2 — Preprocessing (Date→Month + Totals)

Cell 2.1 — Add month column

In [5]:
def add_month_col(df):
    df["date"] = pd.to_datetime(df["date"], format="%d-%m-%Y", errors="coerce")
    df["month"] = df["date"].dt.to_period("M").astype(str)
    return df

bio_df  = add_month_col(bio_df)
demo_df = add_month_col(demo_df)
enr_df  = add_month_col(enr_df)

print("✅ Month column created")
bio_df[["date","month"]].head()

✅ Month column created


,date,month
0,2025-12-28,2025-12
1,2025-12-28,2025-12
2,2025-12-28,2025-12
3,2025-12-28,2025-12
4,2025-12-28,2025-12


Cell 2.2 — Create totals

In [6]:
# Biometric totals
bio_df["bio_updates_total"] = bio_df["bio_age_5_17"].fillna(0) + bio_df["bio_age_17_"].fillna(0)

# Demographic totals
demo_df["demo_updates_total"] = (
    demo_df["age_0_5"].fillna(0) +
    demo_df["age_5_17"].fillna(0) +
    demo_df["age_18_greater"].fillna(0)
)

# Enrolment totals
enr_df["enrolment_total"] = enr_df["demo_age_5_17"].fillna(0) + enr_df["demo_age_17_"].fillna(0)

print("✅ Totals created")

✅ Totals created


SECTION 3 — Aggregate District-Month + Merge Master

Cell 3.1 — Aggregate

In [7]:
bio_agg  = bio_df.groupby(["state","district","month"], as_index=False)["bio_updates_total"].sum()
demo_agg = demo_df.groupby(["state","district","month"], as_index=False)["demo_updates_total"].sum()
enr_agg  = enr_df.groupby(["state","district","month"], as_index=False)["enrolment_total"].sum()

print("✅ Aggregation complete")

✅ Aggregation complete


Cell 3.2 — Merge master

In [8]:
master = pd.merge(enr_agg, demo_agg, on=["state","district","month"], how="outer")
master = pd.merge(master, bio_agg, on=["state","district","month"], how="outer")

for c in ["enrolment_total","demo_updates_total","bio_updates_total"]:
    master[c] = master[c].fillna(0)

print("✅ Master created:", master.shape)
master.head()

✅ Master created: (556, 6)


,state,district,month,enrolment_total,demo_updates_total,bio_updates_total
0,Maharashtra,Ahilyanagar,2025-05,0.0,0.0,22.0
1,Maharashtra,Ahilyanagar,2025-06,0.0,0.0,102.0
2,Maharashtra,Ahilyanagar,2025-07,0.0,0.0,131.0
3,Maharashtra,Ahilyanagar,2025-08,0.0,0.0,149.0
4,Maharashtra,Ahilyanagar,2025-09,0.0,0.0,196.0


SECTION 4 — Filter Maharashtra + Clean District Names

Cell 4.1 — Filter Maharashtra

In [9]:
master_mh = master[master["state"].str.strip().str.lower() == "maharashtra"].copy()
print("✅ Maharashtra rows:", master_mh.shape)
master_mh.head()

✅ Maharashtra rows: (556, 6)


,state,district,month,enrolment_total,demo_updates_total,bio_updates_total
0,Maharashtra,Ahilyanagar,2025-05,0.0,0.0,22.0
1,Maharashtra,Ahilyanagar,2025-06,0.0,0.0,102.0
2,Maharashtra,Ahilyanagar,2025-07,0.0,0.0,131.0
3,Maharashtra,Ahilyanagar,2025-08,0.0,0.0,149.0
4,Maharashtra,Ahilyanagar,2025-09,0.0,0.0,196.0


Cell 4.2 — Clean district names (CRITICAL)

In [10]:
def clean_district_names(df, col="district"):
    df[col] = (
        df[col].astype(str)
        .str.replace(r"[^A-Za-z\s]", "", regex=True)   # remove symbols like *
        .str.replace(r"\s+", " ", regex=True)
        .str.strip()
    )
    return df

master_mh = clean_district_names(master_mh)
print("✅ District cleaned. Unique districts:", master_mh["district"].nunique())

✅ District cleaned. Unique districts: 49


SECTION 5 — Advanced EDA Dashboards + Auto Save

5.1 Maharashtra activity trend (stacked)

In [38]:
# Install necessary Kaleido dependencies
!sudo apt-get update && sudo apt-get install -y libnss3 libatk-bridge2.0-0 libcups2 libxcomposite1 libxdamage1 libxfixes3 libxrandr2 libgbm1 libxkbcommon0 libpango-1.0-0 libcairo2 libasound2


trend = master_mh.groupby("month", as_index=False)[["enrolment_total","demo_updates_total","bio_updates_total"]].sum()
trend["month_dt"] = pd.to_datetime(trend["month"])

fig = go.Figure()
fig.add_trace(go.Scatter(x=trend["month_dt"], y=trend["enrolment_total"], mode="lines+markers", name="Enrolment"))
fig.add_trace(go.Scatter(x=trend["month_dt"], y=trend["demo_updates_total"], mode="lines+markers", name="Demo Updates"))
fig.add_trace(go.Scatter(x=trend["month_dt"], y=trend["bio_updates_total"], mode="lines+markers", name="Bio Updates"))

fig.update_layout(
    title="📊 Maharashtra Aadhaar Activity Trend (Enrolment + Updates)",
    xaxis_title="Month",
    yaxis_title="Count",
    template="plotly_white",
    height=450
)

fig.show()
import kaleido; kaleido.get_chrome_sync()
save_fig(fig, "EDA_Activity_Trend.png")

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://cli.github.com/packages stable InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:5 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading packag

✅ Saved chart: REPORT_CHARTS/EDA_Activity_Trend.png


5.2 Top districts by enrolment

In [14]:
top_dist = master_mh.groupby("district", as_index=False)["enrolment_total"].sum()
top_dist = top_dist.sort_values("enrolment_total", ascending=False).head(20)

fig = px.bar(
    top_dist.sort_values("enrolment_total", ascending=True),
    x="enrolment_total",
    y="district",
    orientation="h",
    text="enrolment_total",
    title="🏆 Top 20 Districts by Total Enrolment (MH)",
    template="plotly_white"
)
fig.update_traces(texttemplate="%{text:.0f}", textposition="outside")
fig.update_layout(height=700)

fig.show()
save_fig(fig, "EDA_Top20_District_Enrolment.png")

✅ Saved chart: REPORT_CHARTS/EDA_Top20_District_Enrolment.png


5.3 Heatmap: district vs month enrolment intensity

In [15]:
heat = master_mh.groupby(["district","month"], as_index=False)["enrolment_total"].sum()
heat["month_dt"] = pd.to_datetime(heat["month"])

pivot = heat.pivot(index="district", columns="month_dt", values="enrolment_total").fillna(0)

fig = px.imshow(
    pivot,
    aspect="auto",
    title="🔥 Heatmap: District vs Month Enrolment Intensity (MH)",
    template="plotly_white"
)
fig.update_layout(height=800)

fig.show()
save_fig(fig, "EDA_Heatmap_Enrolment_District_Month.png")

✅ Saved chart: REPORT_CHARTS/EDA_Heatmap_Enrolment_District_Month.png


SECTION 6 — Feature Engineering + ASSI

6.1 Create features

In [16]:
df_feat = master_mh.copy()
df_feat["month_dt"] = pd.to_datetime(df_feat["month"])
df_feat = df_feat.sort_values(["district","month_dt"])

df_feat["total_updates"] = df_feat["demo_updates_total"] + df_feat["bio_updates_total"]
df_feat["total_transactions"] = df_feat["enrolment_total"] + df_feat["total_updates"]

df_feat["enroll_velocity"] = df_feat.groupby("district")["enrolment_total"].diff().fillna(0)
df_feat["growth_acceleration"] = df_feat.groupby("district")["enroll_velocity"].diff().fillna(0)

prev_updates = df_feat.groupby("district")["total_updates"].shift(1)
df_feat["update_churn"] = (df_feat["total_updates"] / (prev_updates + 1)).fillna(0)

df_feat["center_load_density"] = df_feat["total_transactions"]

print("✅ Features prepared")
df_feat.head()

✅ Features prepared


,state,district,month,enrolment_total,demo_updates_total,bio_updates_total,month_dt,total_updates,total_transactions,enroll_velocity,growth_acceleration,update_churn,center_load_density
0,Maharashtra,Ahilyanagar,2025-05,0.0,0.0,22.0,2025-05-01,22.0,22.0,0.0,0.0,0.000000,22.0
1,Maharashtra,Ahilyanagar,2025-06,0.0,0.0,102.0,2025-06-01,102.0,102.0,0.0,0.0,4.434783,102.0
2,Maharashtra,Ahilyanagar,2025-07,0.0,0.0,131.0,2025-07-01,131.0,131.0,0.0,0.0,1.271845,131.0
3,Maharashtra,Ahilyanagar,2025-08,0.0,0.0,149.0,2025-08-01,149.0,149.0,0.0,0.0,1.128788,149.0
4,Maharashtra,Ahilyanagar,2025-09,0.0,0.0,196.0,2025-09-01,196.0,196.0,0.0,0.0,1.306667,196.0


6.2 Normalize + compute ASSI

In [17]:
stress_cols = ["enroll_velocity","total_updates","growth_acceleration","center_load_density","update_churn"]

scaler = MinMaxScaler()
df_feat[[c+"_norm" for c in stress_cols]] = scaler.fit_transform(df_feat[stress_cols])

weights = {
    "enroll_velocity_norm": 0.25,
    "total_updates_norm": 0.25,
    "growth_acceleration_norm": 0.20,
    "center_load_density_norm": 0.15,
    "update_churn_norm": 0.15
}

df_feat["ASSI_raw"] = (
    weights["enroll_velocity_norm"] * df_feat["enroll_velocity_norm"] +
    weights["total_updates_norm"] * df_feat["total_updates_norm"] +
    weights["growth_acceleration_norm"] * df_feat["growth_acceleration_norm"] +
    weights["center_load_density_norm"] * df_feat["center_load_density_norm"] +
    weights["update_churn_norm"] * df_feat["update_churn_norm"]
)

df_feat["ASSI"] = (df_feat["ASSI_raw"] * 100).round(2)

print("✅ ASSI computed")
df_feat[["district","month","ASSI"]].head()

✅ ASSI computed


,district,month,ASSI
0,Ahilyanagar,2025-05,26.94
1,Ahilyanagar,2025-06,26.97
2,Ahilyanagar,2025-07,26.97
3,Ahilyanagar,2025-08,26.98
4,Ahilyanagar,2025-09,26.99


SECTION 7 — Advanced ASSI Visualizations + Auto Save

7.1 ASSI trend with threshold band

In [18]:
assi_trend = df_feat.groupby("month_dt", as_index=False)["ASSI"].mean()

fig = px.line(
    assi_trend, x="month_dt", y="ASSI", markers=True,
    title="🚨 Maharashtra Mean ASSI Trend (Service Stress Index)",
    template="plotly_white"
)

fig.add_hrect(y0=0, y1=30, opacity=0.12, annotation_text="Stable", line_width=0)
fig.add_hrect(y0=30, y1=60, opacity=0.10, annotation_text="Watchlist", line_width=0)
fig.add_hrect(y0=60, y1=100, opacity=0.08, annotation_text="Critical", line_width=0)

fig.update_layout(height=450)
fig.show()
save_fig(fig, "ASSI_Mean_Trend.png")

✅ Saved chart: REPORT_CHARTS/ASSI_Mean_Trend.png


7.2 Stress zones donut

In [19]:
def stress_category(x):
    if x < 30: return "🟢 Stable"
    elif x < 60: return "🟡 Watchlist"
    else: return "🔴 Critical"

df_feat["StressCategory"] = df_feat["ASSI"].apply(stress_category)

zone_counts = df_feat["StressCategory"].value_counts().reset_index()
zone_counts.columns = ["Zone","Count"]

fig = px.pie(zone_counts, names="Zone", values="Count", hole=0.45,
             title="🧭 Stress Zone Distribution (ASSI Categories)",
             template="plotly_white")

fig.update_layout(height=450)
fig.show()
save_fig(fig, "ASSI_Stress_Zones_Donut.png")

✅ Saved chart: REPORT_CHARTS/ASSI_Stress_Zones_Donut.png


7.3 Heatmap ASSI

In [20]:
df_alert = df_feat.sort_values(["district","month_dt"]).copy()
df_alert["prev_category"] = df_alert.groupby("district")["StressCategory"].shift(1)
df_alert["entered_red"] = ((df_alert["StressCategory"]=="🔴 Critical") &
                           (df_alert["prev_category"]!="🔴 Critical"))

red_entries = df_alert[df_alert["entered_red"]==True][["district","month","ASSI","prev_category","StressCategory"]]
red_entries = red_entries.sort_values("month").tail(30)

fig = go.Figure(data=[go.Table(
    header=dict(values=list(red_entries.columns), fill_color="lightgrey", align="left"),
    cells=dict(values=[red_entries[col] for col in red_entries.columns], align="left")
)])
fig.update_layout(title="🚨 ALERT: Districts Entering RED Zone (Critical Stress)", height=550)
fig.show()

save_fig(fig, "ALERT_RedZone_Entries_Table.png")

✅ Saved chart: REPORT_CHARTS/ALERT_RedZone_Entries_Table.png


SECTION 9 — Labels (Dynamic Threshold ONLY) + Balanced Model

9.1 Dynamic threshold labels

In [21]:
THRESHOLD = df_feat["ASSI"].quantile(0.80)  # top 20% high stress
print("✅ Dynamic threshold used:", THRESHOLD)

df_feat["high_stress"] = (df_feat["ASSI"] >= THRESHOLD).astype(int)
df_feat["HighStressNextMonth"] = df_feat.groupby("district")["high_stress"].shift(-1)

df_model = df_feat.dropna(subset=["HighStressNextMonth"]).copy()
df_model["HighStressNextMonth"] = df_model["HighStressNextMonth"].astype(int)

print("✅ Target distribution:")
print(df_model["HighStressNextMonth"].value_counts())

✅ Dynamic threshold used: 38.75
✅ Target distribution:
HighStressNextMonth
0    400
1    107
Name: count, dtype: int64


9.2 Train final model with sample weights (prevents 0% risk)

In [22]:
X = df_model[stress_cols]
y = df_model["HighStressNextMonth"]

sample_weights = compute_sample_weight(class_weight="balanced", y=y)

final_model = GradientBoostingClassifier(random_state=42)
final_model.fit(X, y, sample_weight=sample_weights)

df_model["risk_probability"] = final_model.predict_proba(X)[:,1]
df_model["month_dt"] = pd.to_datetime(df_model["month"])

print("✅ Model trained and probabilities generated")

✅ Model trained and probabilities generated


SECTION 10 — Risk Prediction Dashboards + Auto Save

10.1 Top 20 districts risk probability

In [23]:
latest_month = df_model["month_dt"].max()
latest_view = df_model[df_model["month_dt"] == latest_month].copy()

latest_view["risk_probability_pct"] = (latest_view["risk_probability"] * 100).round(2)
top_risk = latest_view.sort_values("risk_probability_pct", ascending=False).head(20)

fig = px.bar(
    top_risk.sort_values("risk_probability_pct", ascending=True),
    x="risk_probability_pct",
    y="district",
    orientation="h",
    text="risk_probability_pct",
    title=f"🚨 Predicted Overload Risk Next Month (Top 20 Districts) | Base: {latest_month.strftime('%Y-%m')}",
    template="plotly_white"
)

fig.update_traces(texttemplate="%{text:.2f}%", textposition="outside")
fig.update_layout(height=700, xaxis_title="Risk Probability (%)", yaxis_title="District")
fig.show()
save_fig(fig, "RISK_Top20_Probability.png")

✅ Saved chart: REPORT_CHARTS/RISK_Top20_Probability.png


10.2 Probability distribution histogram (very important)

In [24]:
fig = px.histogram(
    latest_view,
    x="risk_probability_pct",
    nbins=20,
    title=f"📊 Risk Probability Distribution (Latest Month: {latest_month.strftime('%Y-%m')})",
    template="plotly_white"
)
fig.update_layout(height=450, xaxis_title="Risk Probability (%)", yaxis_title="District count")
fig.show()
save_fig(fig, "RISK_Probability_Distribution.png")

✅ Saved chart: REPORT_CHARTS/RISK_Probability_Distribution.png


10.3 Advanced bubble dashboard (risk bands)

In [25]:
latest_view["risk_band"] = pd.cut(
    latest_view["risk_probability_pct"],
    bins=[-1, 30, 70, 101],
    labels=["🟢 Low", "🟡 Medium", "🔴 High"]
)

fig = px.scatter(
    latest_view,
    x="enrolment_total",
    y="total_updates",
    size="ASSI",
    color="risk_band",
    hover_name="district",
    title="🎯 Risk Bands Dashboard: Enrolment vs Updates (size = ASSI)",
    template="plotly_white"
)
fig.update_layout(height=600)
fig.show()
save_fig(fig, "RISK_Bubble_Dashboard.png")

✅ Saved chart: REPORT_CHARTS/RISK_Bubble_Dashboard.png


SECTION 11 — Rolling Forecast Validation (6 months) + Auto Save

11.1 Rolling evaluation

In [26]:
df_model = df_model.sort_values("month_dt").copy()
months_sorted = sorted(df_model["month_dt"].unique())

N_TEST_MONTHS = 6
test_months = months_sorted[-N_TEST_MONTHS:]

results, skipped = [], []

for tm in test_months:
    train_df = df_model[df_model["month_dt"] < tm]
    test_df  = df_model[df_model["month_dt"] == tm]

    if len(test_df) < 5:
        skipped.append((tm.strftime("%Y-%m"), "Too few test samples"))
        continue

    X_train, y_train = train_df[stress_cols], train_df["HighStressNextMonth"]
    X_test, y_test   = test_df[stress_cols], test_df["HighStressNextMonth"]

    if len(y_train.unique()) < 2:
        skipped.append((tm.strftime("%Y-%m"), "Only one class in training"))
        continue

    sw = compute_sample_weight(class_weight="balanced", y=y_train)

    m = GradientBoostingClassifier(random_state=42)
    m.fit(X_train, y_train, sample_weight=sw)

    y_pred = m.predict(X_test)

    results.append({
        "month": tm.strftime("%Y-%m"),
        "samples": len(test_df),
        "accuracy": accuracy_score(y_test, y_pred),
        "precision": precision_score(y_test, y_pred, zero_division=0),
        "recall": recall_score(y_test, y_pred, zero_division=0),
        "f1": f1_score(y_test, y_pred, zero_division=0)
    })

rolling_results = pd.DataFrame(results)

print("✅ Rolling validation complete")
print("Skipped months:", skipped)
rolling_results

✅ Rolling validation complete
Skipped months: [('2026-01', 'Too few test samples')]


,month,samples,accuracy,precision,recall,f1
0,2025-08,52,0.596154,1.000000,0.086957,0.160000
1,2025-09,52,0.807692,1.000000,0.333333,0.500000
2,2025-10,52,0.442308,1.000000,0.093750,0.171429
3,2025-11,52,0.442308,0.147059,1.000000,0.256410
4,2025-12,52,0.307692,0.000000,0.000000,0.000000


11.2 Rolling metrics dashboard

In [27]:
fig = px.line(
    rolling_results,
    x="month",
    y=["accuracy","precision","recall","f1"],
    markers=True,
    title="✅ Rolling Forecast Validation Metrics (Last 6 Months)",
    template="plotly_white"
)
fig.update_layout(height=500, yaxis=dict(range=[0,1]))
fig.show()
save_fig(fig, "MODEL_Rolling_Metrics.png")

✅ Saved chart: REPORT_CHARTS/MODEL_Rolling_Metrics.png


SECTION 12 — Root Cause

In [28]:
imp_df = pd.DataFrame({
    "feature": stress_cols,
    "importance": final_model.feature_importances_
}).sort_values("importance", ascending=False)

fig = px.bar(
    imp_df,
    x="importance",
    y="feature",
    orientation="h",
    title="🧠 Root Cause Analysis: Feature Importance",
    template="plotly_white"
)
fig.update_layout(height=450)
fig.show()
save_fig(fig, "EXPLAIN_Feature_Importance.png")
imp_df

✅ Saved chart: REPORT_CHARTS/EXPLAIN_Feature_Importance.png


,feature,importance
1,total_updates,0.411516
4,update_churn,0.241262
2,growth_acceleration,0.120437
3,center_load_density,0.119262
0,enroll_velocity,0.107523


SECTION 13 — Policy Simulation  

13.1 Simulation function

In [29]:
def simulate_policy(df_in, churn_reduction=0.0, load_reduction=0.0):
    df_sim = df_in.copy()

    df_sim["update_churn"] = df_sim["update_churn"] * (1 - churn_reduction)
    df_sim["center_load_density"] = df_sim["center_load_density"] * (1 - load_reduction)

    # Normalize again
    df_sim[[c+"_norm" for c in stress_cols]] = scaler.transform(df_sim[stress_cols])

    # Recompute ASSI
    df_sim["ASSI_raw"] = (
        weights["enroll_velocity_norm"] * df_sim["enroll_velocity_norm"] +
        weights["total_updates_norm"] * df_sim["total_updates_norm"] +
        weights["growth_acceleration_norm"] * df_sim["growth_acceleration_norm"] +
        weights["center_load_density_norm"] * df_sim["center_load_density_norm"] +
        weights["update_churn_norm"] * df_sim["update_churn_norm"]
    )
    df_sim["ASSI"] = (df_sim["ASSI_raw"] * 100).round(2)
    return df_sim

13.2 Apply scenario

In [30]:
CHURN_REDUCTION = 0.15
LOAD_REDUCTION  = 0.25

df_sim = simulate_policy(df_feat, churn_reduction=CHURN_REDUCTION, load_reduction=LOAD_REDUCTION)
df_sim = clean_district_names(df_sim)

print("✅ Policy applied: churn_reduction=", CHURN_REDUCTION, "load_reduction=", LOAD_REDUCTION)

✅ Policy applied: churn_reduction= 0.15 load_reduction= 0.25


13.3 Build impact table safely (outer merge)

In [31]:
impact = pd.merge(
    df_feat[["district","month","month_dt","ASSI"]],
    df_sim[["district","month","ASSI"]],
    on=["district","month"],
    how="outer",
    suffixes=("_before", "_after")
)

impact["ASSI_before"] = impact["ASSI_before"].fillna(0)
impact["ASSI_after"]  = impact["ASSI_after"].fillna(0)
impact["month_dt"] = pd.to_datetime(impact["month"], errors="coerce")

# risk before & after
df_feat["risk_probability_before"] = final_model.predict_proba(df_feat[stress_cols])[:,1]
df_sim["risk_probability_after"]   = final_model.predict_proba(df_sim[stress_cols])[:,1]

impact = impact.merge(df_feat[["district","month","risk_probability_before"]], on=["district","month"], how="left")
impact = impact.merge(df_sim[["district","month","risk_probability_after"]], on=["district","month"], how="left")

impact["risk_probability_before"] = impact["risk_probability_before"].fillna(0)
impact["risk_probability_after"]  = impact["risk_probability_after"].fillna(0)

print("✅ Impact table ready:", impact.shape)
impact.head()

✅ Impact table ready: (1060, 7)


,district,month,month_dt,ASSI_before,ASSI_after,risk_probability_before,risk_probability_after
0,Ahilyanagar,2025-05,2025-05-01,26.94,26.94,0.010457,0.124676
1,Ahilyanagar,2025-06,2025-06-01,26.97,26.97,0.004029,0.004029
2,Ahilyanagar,2025-07,2025-07-01,26.97,26.97,0.006437,0.009170
3,Ahilyanagar,2025-08,2025-08-01,26.98,26.97,0.006437,0.036496
4,Ahilyanagar,2025-09,2025-09-01,26.99,26.99,0.005784,0.006437


13.4 Policy impact (Stress reduction %) Top 20

In [32]:
latest_month_pol = impact["month_dt"].max()
impact_latest = impact[impact["month_dt"]==latest_month_pol].copy()

impact_latest["assi_reduction"] = (impact_latest["ASSI_before"] - impact_latest["ASSI_after"]).round(3)
impact_latest["assi_reduction_pct"] = np.where(
    impact_latest["ASSI_before"] > 0,
    (impact_latest["assi_reduction"] / impact_latest["ASSI_before"]) * 100,
    0
).round(2)

stress_reduced = impact_latest[impact_latest["assi_reduction"] > 0].copy()
stress_reduced = stress_reduced.sort_values("assi_reduction_pct", ascending=False).head(20)

fig = px.bar(
    stress_reduced.sort_values("assi_reduction_pct", ascending=True),
    x="assi_reduction_pct",
    y="district",
    orientation="h",
    text="assi_reduction_pct",
    title=f"✅ Policy Impact: Top 20 Districts by ASSI Stress Reduction (%) | {latest_month_pol.strftime('%Y-%m')}",
    template="plotly_white"
)
fig.update_traces(texttemplate="%{text:.2f}%", textposition="outside")
fig.update_layout(height=700, xaxis_title="Stress Reduction (%)")
fig.show()
save_fig(fig, "POLICY_Stress_Reduction_Top20.png")

✅ Saved chart: REPORT_CHARTS/POLICY_Stress_Reduction_Top20.png


13.5 Policy impact (Risk Δ%) Top 20

In [33]:
impact_latest["risk_before_pct"] = (impact_latest["risk_probability_before"] * 100).round(4)
impact_latest["risk_after_pct"]  = (impact_latest["risk_probability_after"] * 100).round(4)
impact_latest["risk_delta_pct"]  = (impact_latest["risk_before_pct"] - impact_latest["risk_after_pct"]).round(4)

risk_reduced = impact_latest[impact_latest["risk_delta_pct"] > 0].copy()
risk_reduced = risk_reduced.sort_values("risk_delta_pct", ascending=False).head(20)

fig = px.bar(
    risk_reduced.sort_values("risk_delta_pct", ascending=True),
    x="risk_delta_pct",
    y="district",
    orientation="h",
    text="risk_delta_pct",
    title=f"🚨 Policy Impact: Top 20 Districts by Risk Reduction (Probability Δ %) | {latest_month_pol.strftime('%Y-%m')}",
    template="plotly_white"
)
fig.update_traces(texttemplate="%{text:.3f}%", textposition="outside")
fig.update_layout(height=700, xaxis_title="Risk Reduction (Probability Delta % points)")
fig.show()
save_fig(fig, "POLICY_Risk_Reduction_Top20.png")

✅ Saved chart: REPORT_CHARTS/POLICY_Risk_Reduction_Top20.png


13.6 Before vs After heatmaps (ASSI)

In [34]:
top_dists = df_feat.groupby("district")["ASSI"].max().sort_values(ascending=False).head(15).index.tolist()

before_h = df_feat[df_feat["district"].isin(top_dists)].pivot_table(index="district", columns="month", values="ASSI", aggfunc="mean").fillna(0)
after_h  = df_sim[df_sim["district"].isin(top_dists)].pivot_table(index="district", columns="month", values="ASSI", aggfunc="mean").fillna(0)

fig1 = px.imshow(before_h, aspect="auto", title="🔥 BEFORE Policy: ASSI Heatmap (Top 15 stressed districts)", template="plotly_white")
fig1.update_layout(height=650)
fig1.show()
save_fig(fig1, "POLICY_ASSI_Heatmap_BEFORE.png")

fig2 = px.imshow(after_h, aspect="auto", title="✅ AFTER Policy: ASSI Heatmap (Top 15 stressed districts)", template="plotly_white")
fig2.update_layout(height=650)
fig2.show()
save_fig(fig2, "POLICY_ASSI_Heatmap_AFTER.png")

✅ Saved chart: REPORT_CHARTS/POLICY_ASSI_Heatmap_BEFORE.png


✅ Saved chart: REPORT_CHARTS/POLICY_ASSI_Heatmap_AFTER.png


13.7 Final policy recommendation table + save

In [35]:
policy_table = risk_reduced[[
    "district","ASSI_before","ASSI_after","assi_reduction_pct","risk_before_pct","risk_after_pct","risk_delta_pct"
]].copy()

policy_table.columns = [
    "District","ASSI Before","ASSI After","Stress Reduction (%)",
    "Risk Before (%)","Risk After (%)","Risk Delta (%)"
]

fig = go.Figure(data=[go.Table(
    header=dict(values=list(policy_table.columns), fill_color="lightgrey", align="left"),
    cells=dict(values=[policy_table[col] for col in policy_table.columns], align="left")
)])
fig.update_layout(title="🏛 Policy Recommendation Table (Top Beneficiary Districts)", height=600)
fig.show()
save_fig(fig, "POLICY_Recommendation_Table.png")

policy_table.to_csv("MH_policy_impact_top20.csv", index=False)
print("✅ Exported: MH_policy_impact_top20.csv")

✅ Saved chart: REPORT_CHARTS/POLICY_Recommendation_Table.png
✅ Exported: MH_policy_impact_top20.csv


SECTION 14 — Final Exports

In [36]:
df_feat.to_csv("MH_final_ASSI_full.csv", index=False)
df_model.to_csv("MH_risk_predictions_full.csv", index=False)
df_sim.to_csv("MH_simulated_ASSI_full.csv", index=False)
impact.to_csv("MH_policy_impact_full.csv", index=False)

print("✅ Final Exported Files:")
print("1) MH_final_ASSI_full.csv")
print("2) MH_risk_predictions_full.csv")
print("3) MH_simulated_ASSI_full.csv")
print("4) MH_policy_impact_full.csv")
print(f"✅ All charts saved in folder: {SAVE_DIR}/")

✅ Final Exported Files:
1) MH_final_ASSI_full.csv
2) MH_risk_predictions_full.csv
3) MH_simulated_ASSI_full.csv
4) MH_policy_impact_full.csv
✅ All charts saved in folder: REPORT_CHARTS/
